In [92]:
import pprint
import datetime
import time
from pymongo import MongoClient

client = MongoClient('192.168.0.29:27017')
db = client['test']

def do_aggregate(collection, query):
    cursor = collection.aggregate(query)
    return [x for x in cursor]


In [ ]:
# 1. Total record count
print 'Record count'
print db.liverpool.count()
print '\n'

In [ ]:
# 2. Count of nodes/ways
print 'Node/way element frequency'
pipeline = [{'$match':{'$or':[{'type':"way"},{'type':"node"}]}},
             {'$group': {'_id': '$type', 'count': {'$sum':1}}},
             {"$limit": 2}]

agg1 = do_aggregate(db.liverpool, pipeline)
for x in agg1:
    print x['_id'], '\t', x['count']
print '\n'
   

In [ ]:
 
# 3. Unique users    
print 'Unique users:'
print len(db.liverpool.distinct('created.user'))
print '\n'

In [ ]:
# 4. Top 5 contributors as percentage of contributions
totalUserPosts=db.liverpool.count({"created.user": {"$exists": True}})
pipeline = [{"$match": {"created.user": { "$exists": True }}},
            {"$group":{"_id":"$created.user","count":{"$sum":1}}},
            {"$project":
                {"count":1,"percentage":{"$multiply":[{"$divide":[100,totalUserPosts]},"$count"]}}
            },
            {"$sort" : {"count": -1}},
            {"$limit": 5}]

    
agg2 = do_aggregate(db.liverpool, pipeline)
print 'Users by contribution percentage'
for x in agg2:
    print x['_id'], x['count'], "%.2f" % x['percentage']
print '\n'

In [131]:
# 5. Top 10 months for contributions 
pipeline = [{"$match": {"created.user": { "$exists": True }}},
            {'$project':
                {'username': '$created.user',
                'year': { '$year': "$created.timestamp" },
                'month': { '$month': "$created.timestamp" }
                }},
            {'$group':{'_id': {'year':'$year', 'month':'$month', 'username':'$username'}, 'count':{'$sum':1}}},
            {'$sort': {'count':-1}},
            {'$limit': 10}]
agg3 = do_aggregate(db.liverpool, pipeline)
print 'Top 10 months for contributions'
for x in agg3:
    print '%s\t%s\t%s\t%d' % (x['_id']['year'], x['_id']['month'], x['_id']['username'],  x['count'])

Record count
1436842


Node/way element frequency
way 	242005
node 	1194827


Unique users:
693


Users by contribution percentage
daviesp12 1066921 74.25
jrdx 36075 2.51
UniEagle 34564 2.41
F1rst_Timer 22301 1.55
duxxa 20953 1.46




In [157]:
# Postcode re-assessment
import re
postcode_re = re.compile(r'^(GIR ?0AA|[A-PR-UWYZ]([0-9]{1,2}|([A-HK-Y][0-9]([0-9ABEHMNPRV-Y])?)|[0-9][A-HJKPS-UW]) ?[0-9][ABD-HJLNP-UW-Z]{2})$')
pipeline = [{'$match': {'address.postcode':{'$exists':True}, }},
            {'$match': {'address.postcode':{'$not': postcode_re}}},
            {'$project': {'postcode':'$address.postcode'}},
            {'$group': {'_id': '$postcode', 'count':{'$sum':1}}},
            {'$project': {'postcode':'$_id', 'count':'$count'}},
            {'$limit': 20}]
            
res = do_aggregate(db.liverpool,pipeline)
print 'Invalid postcodes'
for x in res:
    print x['postcode'], x['count']

L35 9JY. 1
L1 4LN,L1 3DN 3
L1 1
CH43 1
L17 1
CH63 3H 1
L18 1


In [174]:
res = do_aggregate(db.liverpool, 
                   [{'$match':{'address.postcode':{'$exists':True}}},
                    {'$project':{'postcode':'$address.postcode'}}])
prefix = set()
print 'Total postcodes: %d' % len(res)
for x in res:
    prefix.add( x['postcode'].split(' ')[0])
print 'Unique postcode prefixes'
pprint.pprint(prefix)


Total postcodes: 14258

set([u'CH41',
     u'CH42',
     u'CH421RT',
     u'CH43',
     u'CH439UL',
     u'CH44',
     u'CH45',
     u'CH46',
     u'CH47',
     u'CH48',
     u'CH49',
     u'CH60',
     u'CH62',
     u'CH620DB',
     u'CH63',
     u'CH64',
     u'CH65',
     u'CH66',
     u'L1',
     u'L13',
     u'L15',
     u'L16',
     u'L17',
     u'L18',
     u'L189UD',
     u'L19',
     u'L2',
     u'L20',
     u'L22',
     u'L23',
     u'L24',
     u'L25',
     u'L26',
     u'L27',
     u'L29',
     u'L3',
     u'L30',
     u'L32',
     u'L33',
     u'L34',
     u'L35',
     u'L36',
     u'L4',
     u'L5',
     u'L6',
     u'L69',
     u'L7',
     u'L8',
     u'L9',
     u'WA10',
     u'WA11',
     u'WA7',
     u'WA8'])
